In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('chat_gpt_context/distilbert_squad_dataset.csv')
df = df[['question', 'context', 'answer']]
df.head()

,question,context,answer
0,GET /vulnerabilities/sqli/?id=%28SELECT%20%28C...,If any string appears between select and from ...,SQLInjection
1,GET /bbs/board.php?bo_table=ad_04_01%27)%20AND...,If any string appears between select and from ...,SQLInjection
2,"ataroom/notice_list.asp"" class=""depth1_btn"">.?...",If any string appears between select and from ...,SQLInjection
3,GET /plus/recommend.php?action=&aid=1&_FILES[t...,If any string appears between select and from ...,SQLInjection
4,DELETE /vizql/w/_01___v3DB/v/__6/destroy/sessi...,If any string appears between select and from ...,SQLInjection


In [3]:
# answer_start, answer_end 계산
df['answer_start'] = df.apply(lambda row: row['context'].find(row['answer']), axis=1)
df['answer_end'] = df.apply(lambda row: row['answer_start'] + len(row['answer']), axis=1)

In [4]:
df.head()

,question,context,answer,answer_start,answer_end
0,GET /vulnerabilities/sqli/?id=%28SELECT%20%28C...,If any string appears between select and from ...,SQLInjection,129,141
1,GET /bbs/board.php?bo_table=ad_04_01%27)%20AND...,If any string appears between select and from ...,SQLInjection,129,141
2,"ataroom/notice_list.asp"" class=""depth1_btn"">.?...",If any string appears between select and from ...,SQLInjection,129,141
3,GET /plus/recommend.php?action=&aid=1&_FILES[t...,If any string appears between select and from ...,SQLInjection,129,141
4,DELETE /vizql/w/_01___v3DB/v/__6/destroy/sessi...,If any string appears between select and from ...,SQLInjection,129,141


In [5]:
df['answer'].value_counts()

AttemptAccessAdminPage    11722
CrossSiteScripting         1910
WordPress                  1184
SQLInjection                903
maliciousbot                673
CommandInjection            396
JNDIInjection                89
Name: answer, dtype: int64

In [6]:
df['question'] = df['question'].str.lower()
df['context'] = df['context'].str.lower()
df['answer'] = df['answer'].str.lower()

In [7]:
import json

# JSON으로 변환
json_data = json.dumps({
    'data': [
        {
            'paragraphs': [
                {
                    'context': row['context'],
                    'qas': [
                        {
                            'question': row['question'],
                            'id': str(idx),
                            'answers': [
                                {
                                    'text': row['answer'],
                                    'answer_start': row['answer_start'],
                                    'answer_end': row['answer_end']
                                }
                            ]
                        }
                    ]
                }
            ]
        } for idx, row in df.iterrows()
    ]
})

In [8]:
# json_data

In [9]:
file_path = 'chat_gpt_context'
file_name = 'distilbert_squad_dataset.json'

In [10]:
import os
# JSON 파일 저장
# with open(os.path.join(file_path, file_name), 'w', encoding='utf-8') as f:
#   f.write(json_data)